In [12]:
import numpy as np
import xarray as xr
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

In [13]:
# Forcing profiles Linear and Tanh

# Linear
def ST_linear_profile(ST_surface, ST_bottom, z, z_bottom):
    """
    ST_surface = Salinity or Temperature at z = 0 
    ST_bottom = Salinity or Temperature at z_bottom
    z = z levels
    z_bottom = deepest z level
    """
    ST_z = ST_surface + (ST_bottom - ST_surface)*(z/z_bottom)
    return ST_z

# Tanh
def T_tanh_profile(T_surface, T_bottom, S_surface, z, z_bottom, alpha, beta, rho0):
    """ 
    T_surface = Temperature at z = 0 
    T_bottom = Temperature at z_bottom
    S_surface = Salinity at z = 0
    z = z levels
    z_bottom = deepest z level
    alpha =  3.733e-5 [laddie/plume] / 7.5e-5 [PICO] # [1/degC]    Thermal expansion coefficient
    beta  =  7.843e-4 [laddie/plume] / 7.7e-4 [PICO] # [1/psu]     Haline contraction coefficient
    rho0  =  1028.    [laddie/plume] / 1033   [PICO] # [kg/m^3]    Reference density of seawater  
    """
    drho      = .01*np.abs(z)**.5

    T_z = T_bottom + (T_surface - T_bottom) * (1+np.tanh((z-z_bottom)/250))/2
    S_z = S_surface + alpha*(T_z-T_surface)/beta + drho/(beta*rho0)
    return T_z, S_z


# Equation of state (density anomaly)
def Density_anomaly(T, S,alpha,beta,rho0):
    """ 
    alpha =  3.733e-5 [laddie/plume] / 7.5e-5 [PICO] # [1/degC]    Thermal expansion coefficient
    beta  =  7.843e-4 [laddie/plume] / 7.7e-4 [PICO] # [1/psu]     Haline contraction coefficient
    rho0  =  1028.    [laddie/plume] / 1033   [PICO] # [kg/m^3]    Reference density of seawater  
    """

    sig0 = rho0-1000

    T0 = -1.9
    S0 = 34.0

    sig = sig0 + 1000 * (-alpha*(T-T0)+beta*(S-S0))

    return sig 

In [14]:
z_space = np.linspace(-720,0,73)

# Tanh profiles
T_tanh_COLD,    S_tanh_COLD     = T_tanh_profile(     -1.9,    -1.9,  34,   z_space,    -450, 3.733e-5, 7.843e-4,  1028)
T_tanh_MEDIUM,  S_tanh_MEDIUM   = T_tanh_profile(     -1.9,     0.3,  34,   z_space,    -450, 3.733e-5, 7.843e-4,  1028)
T_tanh_WARM,    S_tanh_WARM     = T_tanh_profile(     -1.9,     1.5,  34,   z_space,    -450, 3.733e-5, 7.843e-4,  1028)

In [ ]:
mpl.rcParams['font.size'] = 13
mpl.rcParams['axes.labelsize'] = 13  # Set the fontsize for axis labels

fig,ax = plt.subplots(1,2,figsize=(7,4), dpi=300,sharey=True)

# Temperature
ax[0].set_xlabel(r'Temperature [$\degree$C]')
ax[0].set_ylabel(r'Depth [m]')
ax[0].set_ylim(-720,0)

ax[0].plot(T_tanh_MEDIUM,   z_space, color='darkorange',    lw=4,    label='moderate-melt')
ax[0].plot(T_tanh_WARM,     z_space, color='darkred',       lw=4,    label='high-melt')

ax[0].grid(True,alpha=0.3,ls='dashed')

# Salinity
ax[1].set_xlabel(r'Salinity [psu]')
ax[1].set_ylim(-720,0)

ax[1].plot(S_tanh_MEDIUM,   z_space, color='darkorange',    lw=4,    label='moderate')
ax[1].plot(S_tanh_WARM,     z_space, color='darkred',       lw=4,    label='high-melt')

ax[1].grid(True,alpha=0.3,ls='dashed')

ax[0].legend()

# Pannel labels
ax[0].text(0.05, 0.05, '(a)', transform=ax[0].transAxes, fontsize=13, weight='bold',  ha='left')
ax[1].text(0.05, 0.05, '(b)', transform=ax[1].transAxes, fontsize=13, weight='bold',  ha='left')


fig.tight_layout()
fig.savefig('fig03.pdf',dpi=300)
